In [13]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_datareader.famafrench import get_available_datasets,FamaFrenchReader
import pandas_datareader.data as web
import warnings
import statsmodels.api as sm


warnings.simplefilter(action="ignore", category=FutureWarning)

tickers = [
    'DIS', 'UHS', 'GOOGL', 'LOW', 'BSX', 'JPM', 'EOG', 'SBAC', 'STT', 'DVN',
    'VRTX', 'BK', 'ALL', 'WY', 'OXY', 'ON', 'EXC', 'UAL', 'AKAM', 'SMCI',
    'LYV', 'AMD', 'VLO', 'NI', 'DRI', 'SJM', 'APH', 'MAS', 'CMG', 'TXT'
]

n_tickers = len(tickers)

start_date = '2014-01-01'
end_date = '2023-12-31'

# Download the daily data from stocks
price = yf.download(tickers, start=start_date, end=end_date, progress=False)['Adj Close']
daily_returns_stocks = price.pct_change()
daily_returns_stocks = daily_returns_stocks.dropna()

# Download factors. All data is monthly, so it is necessary to change the returns to be monthly returns
research_factors = FamaFrenchReader("F-F_Research_Data_5_Factors_2x3_daily",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0]
industry_factors = FamaFrenchReader("5_Industry_Portfolios_daily",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0] # Average Value Weighted Returns -- Monthly
momentum_factor = FamaFrenchReader("F-F_Momentum_Factor_daily",start=pd.to_datetime(start_date),end=pd.to_datetime(end_date)).read()[0]

# SMB -> Factor for Size
# HML -> Factor for B/M
# RNW -> Factor for profitabilty
# CMA -> Factor for growth
# Mom -> Factor for Momentum
# Mkt -> Factor for Mkt

# Merge all factors
factors = pd.concat([research_factors,industry_factors,momentum_factor],axis=1)
factors["Mkt"] = factors["Mkt-RF"] + factors["RF"]
factors = factors.drop(columns=["Mkt-RF","RF"])

factors = factors.loc[:].div(100)
factors = factors.iloc[1:,:]

In [14]:
# This function uses returns as the target variable and the factors as features. We want to predict the returns using the factors.
# result.params returns the beta used to model
def fit_factor_model(returns, factors):
    result = sm.OLS(returns, factors).fit()
    return result.params

# Creating a matrix with all the betas of each factor
# Beta_i is the sensitivity of the asset i return to the factor
# I_x is the factor value 
# Mean Return for asset i can be expressed as Beta_i1*mean_I_1 + Beta_i2*mean_I_2 ...
betas = pd.DataFrame(index=daily_returns_stocks.columns, columns = factors.columns)
for stock in daily_returns_stocks.columns:
    betas.loc[stock] = fit_factor_model(daily_returns_stocks[stock], factors)
    
betas

,SMB,HML,RMW,CMA,Cnsmr,Manuf,HiTec,Hlth,Other,Mom,Mkt
Ticker,,,,,,,,,,,
AKAM,-0.083611,-0.36246,0.00992,0.512994,0.714276,0.420416,1.412412,0.53217,0.803572,-0.024206,-2.965014
ALL,-0.194168,0.174733,0.12501,0.044131,0.496729,0.422454,0.351047,0.326398,1.046559,0.094909,-1.748197
AMD,0.38769,-0.320542,-0.195033,0.174705,-1.183703,-0.779962,-0.282392,-0.886316,-1.420268,-0.018721,5.853245
APH,0.198771,-0.343527,0.288006,0.083385,0.039339,0.520077,0.643346,0.015952,0.683923,0.002949,-0.925416
BK,-0.157067,0.540895,-0.258776,-0.051893,0.14027,-0.273929,0.179014,0.178701,1.240803,-0.073261,-0.497326
BSX,0.037611,-0.260665,0.233734,-0.066933,0.632248,0.952309,1.441659,1.045899,1.428874,0.031012,-4.461808
CMG,0.222655,-0.484678,0.026775,-0.083422,-0.304561,-0.79262,-1.29023,-0.718414,-0.325702,0.066522,4.208983
DIS,-0.024981,-0.197944,-0.15561,0.031887,0.921856,0.733105,1.578598,0.176862,1.657326,-0.208486,-4.188444
DRI,0.481056,-0.520297,0.60495,-0.081337,0.531175,0.296425,0.08955,-0.166564,1.779126,-0.001005,-1.519519


In [15]:
factor_cov_matrix = factors.cov()
factor_cov_matrix

,SMB,HML,RMW,CMA,Cnsmr,Manuf,HiTec,Hlth,Other,Mom,Mkt
SMB,4.347282e-05,0.000014,-0.000009,1.125604e-07,0.000011,2.106380e-05,0.000008,0.000011,0.000025,-0.000021,0.000015
HML,1.382259e-05,0.000077,0.000015,2.480226e-05,-0.000014,2.817725e-05,-0.000031,-0.000019,0.000026,-0.000030,-0.000006
RMW,-8.774132e-06,0.000015,0.000025,7.372107e-06,-0.000008,-4.539227e-06,-0.000014,-0.000015,-0.000008,-0.000002,-0.000011
CMA,1.125604e-07,0.000025,0.000007,2.042385e-05,-0.000017,6.161798e-08,-0.000027,-0.000014,-0.000009,0.000002,-0.000016
Cnsmr,1.091934e-05,-0.000014,-0.000008,-1.695115e-05,0.000119,1.014645e-04,0.000130,0.000091,0.000117,-0.000013,0.000117
Manuf,2.106380e-05,0.000028,-0.000005,6.161798e-08,0.000101,1.414960e-04,0.000118,0.000094,0.000137,-0.000033,0.000121
HiTec,8.014321e-06,-0.000031,-0.000014,-2.706455e-05,0.000130,1.183845e-04,0.000179,0.000114,0.000140,-0.000010,0.000146
Hlth,1.113195e-05,-0.000019,-0.000015,-1.369140e-05,0.000091,9.355706e-05,0.000114,0.000124,0.000107,-0.000005,0.000106
Other,2.477794e-05,0.000026,-0.000008,-9.483214e-06,0.000117,1.369302e-04,0.000140,0.000107,0.000167,-0.000035,0.000138
Mom,-2.128077e-05,-0.000030,-0.000002,2.008126e-06,-0.000013,-3.335998e-05,-0.000010,-0.000005,-0.000035,0.000118,-0.000019


In [16]:
specific_variances = pd.Series(index=daily_returns_stocks.columns)
# factors.dot(betas.loc[stock]) -> I_1*Beta_i1
# This for is to get the idiosyncratic risk for each stock
for stock in daily_returns_stocks.columns:
    residuals = daily_returns_stocks[stock] - factors.dot(betas.loc[stock])
    specific_variances[stock] = residuals.var()
    
estimate_with_factors_cov_matrix = betas.dot(factor_cov_matrix).dot(betas.T) + pd.DataFrame(np.diag(specific_variances), index=daily_returns_stocks.columns, columns=daily_returns_stocks.columns)

In [17]:
estimate_with_factors_cov_matrix

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,...,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,
AKAM,0.00039,0.000076,0.000194,0.000114,0.000093,0.000109,0.000103,0.000103,0.000105,0.000115,...,0.000095,0.000045,0.000137,0.000117,0.00012,0.000125,0.000104,0.000097,0.000128,0.000122
ALL,0.000076,0.000221,0.000117,0.000108,0.00014,0.000105,0.000074,0.000109,0.000136,0.000188,...,0.000076,0.00005,0.000121,0.000166,0.000159,0.00018,0.000131,0.000159,0.000089,0.000144
AMD,0.000194,0.000117,0.001342,0.000218,0.000164,0.000173,0.000219,0.000199,0.000198,0.000257,...,0.00016,0.000042,0.000295,0.000216,0.000224,0.000246,0.000168,0.000176,0.000189,0.000223
APH,0.000114,0.000108,0.000218,0.000216,0.000141,0.000129,0.00012,0.000139,0.000159,0.000213,...,0.000103,0.000046,0.000179,0.000177,0.000184,0.000204,0.000146,0.000167,0.000119,0.000172
BK,0.000093,0.00014,0.000164,0.000141,0.000291,0.000131,0.000102,0.000153,0.000186,0.000255,...,0.000087,0.000043,0.000169,0.000239,0.000213,0.000268,0.000173,0.000209,0.000107,0.000185
BSX,0.000109,0.000105,0.000173,0.000129,0.000131,0.000293,0.000096,0.000119,0.000137,0.00017,...,0.000101,0.000052,0.00015,0.000159,0.000161,0.000171,0.000148,0.000151,0.000151,0.000152
CMG,0.000103,0.000074,0.000219,0.00012,0.000102,0.000096,0.000477,0.000114,0.000129,0.000117,...,0.000086,0.000026,0.000158,0.000131,0.000127,0.000159,0.000103,0.00009,0.000096,0.00013
DIS,0.000103,0.000109,0.000199,0.000139,0.000153,0.000119,0.000114,0.000293,0.000162,0.000212,...,0.000094,0.000039,0.000167,0.000188,0.000183,0.000222,0.000143,0.000167,0.0001,0.000168
DRI,0.000105,0.000136,0.000198,0.000159,0.000186,0.000137,0.000129,0.000162,0.000492,0.000237,...,0.000095,0.000048,0.000191,0.000227,0.000224,0.000285,0.000184,0.000199,0.0001,0.000205


In [18]:
regular_matrix_cov = daily_returns_stocks.cov()
regular_matrix_cov

Ticker,AKAM,ALL,AMD,APH,BK,BSX,CMG,DIS,DRI,DVN,...,SBAC,SJM,SMCI,STT,TXT,UAL,UHS,VLO,VRTX,WY
Ticker,,,,,,,,,,,,,,,,,,,,,
AKAM,0.000390,0.000077,0.000189,0.000118,0.000101,0.000102,0.000096,0.000100,0.000071,0.000125,...,0.000114,0.000062,0.000131,0.000116,0.000121,0.000106,0.000089,0.000090,0.000142,0.000108
ALL,0.000077,0.000221,0.000107,0.000111,0.000138,0.000107,0.000080,0.000105,0.000141,0.000175,...,0.000082,0.000058,0.000108,0.000163,0.000163,0.000174,0.000135,0.000156,0.000079,0.000141
AMD,0.000189,0.000107,0.001340,0.000222,0.000172,0.000173,0.000223,0.000179,0.000179,0.000279,...,0.000153,0.000040,0.000318,0.000233,0.000216,0.000227,0.000154,0.000190,0.000202,0.000219
APH,0.000118,0.000111,0.000222,0.000216,0.000144,0.000132,0.000122,0.000130,0.000159,0.000200,...,0.000104,0.000044,0.000184,0.000180,0.000193,0.000214,0.000147,0.000162,0.000108,0.000172
BK,0.000101,0.000138,0.000172,0.000144,0.000291,0.000128,0.000094,0.000151,0.000175,0.000258,...,0.000081,0.000045,0.000167,0.000299,0.000214,0.000248,0.000167,0.000208,0.000109,0.000176
BSX,0.000102,0.000107,0.000173,0.000132,0.000128,0.000292,0.000111,0.000123,0.000159,0.000168,...,0.000103,0.000053,0.000153,0.000156,0.000162,0.000193,0.000179,0.000164,0.000141,0.000156
CMG,0.000096,0.000080,0.000223,0.000122,0.000094,0.000111,0.000477,0.000119,0.000194,0.000160,...,0.000088,0.000009,0.000116,0.000120,0.000106,0.000178,0.000124,0.000096,0.000099,0.000129
DIS,0.000100,0.000105,0.000179,0.000130,0.000151,0.000123,0.000119,0.000294,0.000190,0.000225,...,0.000090,0.000034,0.000160,0.000186,0.000191,0.000251,0.000149,0.000171,0.000100,0.000167
DRI,0.000071,0.000141,0.000179,0.000159,0.000175,0.000159,0.000194,0.000190,0.000491,0.000275,...,0.000089,0.000034,0.000173,0.000214,0.000231,0.000358,0.000237,0.000240,0.000098,0.000221


### With the covariance matrix, it is possible to calculate the risk of the portfolio. With the risk of the portfolio and also the expected return of the portfolio we can calculate sharpe ratio and decide if the factor model or the normal model were better